# Introdução à biblioteca BioPython

Nesse exemplo, usamos as bibliotecas Biopython, Pandas e Matplotlib para explorar e gerar visualizações dos dados de um genoma bacteriano.

## 1. Importando bibliotecas

As bibliotecas de Python que usaremos neste exercício são:

* numpy      : biblioteca para computação científica
* pandas     : biblioteca base para análise manipulação e análise de dados
* Bio        : Biopython (base)
* matplotlib : biblioteca para preparação de gráficos e outras representações visuais

Abaixo, a linha `%matplolib inline` informa que os gráficos da biblioteca matplotlib devem ser exibidos dentro do Jupyter e sem a necessidade de chamadas explícitas para o método `plt.show()`.

In [ ]:
import Bio
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

No caso específico das análises propostas nesse tutorial, será mais prático importar os módulos de BioPython que implementam algumas rotinas de processamento e análise de sequências:

In [ ]:
from Bio import SeqIO
from Bio import SeqUtils

## 2. Definindo o caminho dos dados

Começamos carregando os contigs do genoma de *Klebsiella pneumoniae*, geradodos na aula de montagem de genomas ([Aula 4](https://edisciplinas.usp.br/course/view.php?id=79135#section-4)). Esse arquivo contém as sequências de 1788 contigs, de comprimentos diferentes, que serão processados usando a biblioteca BioPython. Os resultados desse processamento serão armazendos em um dataframe Pandas para processamento posterior.

## 3. Carregando as sequências (BioPython)

Começamos abrindo o arquivo FASTA para que o BioPython possa processá-lo.

In [ ]:
io = SeqIO.parse('kpneu-contigs.fa',"fasta")

e, em seguida, carregamos as sequências contidas no arquivo em uma lista:

In [ ]:
s = []
for x in io:
    s.append(x)

## 4. Explorando as sequências com BioPython

Se o código acima funcionou, a variável `s` é uma lista e deve conter objetos que representam todas as sequências de contigs do genoma de _K. pneumoniae_ strain KPNIH39 (isolado SAMN03455930). Para verificar se nosso código, até esse ponto, deu certo, vamos imprimmir algumas informações sobre `s` e seu conteúdo.

Comecemos verificando o número de elementos na lista `s`:

In [ ]:
len(s)

É sempre importante verificar se o resultado produzido por uma nova linha de código correponde ao esperado. Podemos fazer isso usando um comando externo, como _grep_:

`grep -c ">" kpneu-contigs.fa`

que já vimos na [aula de Linux](https://edisciplinas.usp.br/course/view.php?id=79135#section-1) e que vai contar o número de sequências no arquivo de contigs.

Execute o comando acima e veja se obtem o mesmo resultado.

Além da verificação acima, vamos tentar visualizar o dado armazenado em um elemento de `s`.

Comecemos imprimindo (*i.e.* representando como texto) __o objeto__ que representa a primeira sequência do arquivo:

In [ ]:
s[0]

Então checamos o comprimento da sequência:

In [ ]:
len(s[0])

E, já que a sequência é curta, inspecionamos a própria sequência:

In [ ]:
str(s[0].seq)

e comparamos, por inspeção, o resultado acima com a segunda linha do arquivo original, que podemos ver, na linha de comando do Linux, com o comando

`less -S /databases/courses/bmp0216/2020/similaridade/kpneu-contigs.fa`

Podemos testar alguns atributos e métodos da classe SeqRecord, como checar o identificador da primeira sequência:

In [ ]:
s[0].id

Para ilustrar as funções disponíveis no BioPython podemos, por exemplo, traduzir os 27 primeiros nucleotídeos da sequência `s[0]` para proteína:

In [ ]:
str(s[0][0:27].translate(table=11).seq)

O parâmetro `table=11` passado ao método `translate()` seleciona o código genético usado pelas bactérias, arqueas, mitocôndrias e cloroplastos.

Note que os 27 primeiros resíduos da sequência são extraídos usando a notação de Python para listas e *strings* (`[0:27]`), aplicada sobre o objeto `s[0]`. Embora essa notação tenha sido introduzida anteriormente apenas para listas e texto (*strings*), o objeto retornado por esse código (`s[0][0:27]`), tal qual o próprio `s[0]`, é um Bio.SeqRecord, e portanto suporta o método `translate()`.

## 5. Explorando e analisando estatísticas de contigs com Biopython e Pandas

Para entender melhor os dados gerados pela montagem, podemos calcular e avaliar algumas estatísticas descritivas sobre os contigs.

### 5.1 Histogramas com Biopython e Pandas.Series

Usando uma _list conprehension_ podemos calcular o comprimento de todos os contigs mas, desta vez, armazenaremos o resultado em uma série da biblioteca Pandas:

In [ ]:
clen = pd.Series([len(x) for x in s])

No cálculo acima usamos a sintaxe conhecida como __*list comprehension*__ (`[ len(x) for x in s ]`) para criar a lista que convertemos em série.

Esse código compacto é equivalente a

>`a = []
for x in s:
    a.append(len(x))
clen = pd.Series(a)
del(a)` 

Vamos ver o que foi armazenado em `clen`:

In [ ]:
clen

```
Nota: a visualização acima está truncada ("..." indica o truncamento).
No entanto, nosso código funcionou pois o total de elementos na série (1788) é igual ao número de contigs.
```

Vejamos como fica a distribuição de comprimentos de contigs!

Primeiro, ajustamos o tamanho das figuras para este *notebook* usando o código `mpl.rc('figure',figsize=(12.8,4.8))`.

Em seguida chamamos o método `hist()` da pandas.Series (`clen`) que contém os comprimentos de cada *contig*.

In [ ]:
%matplotlib inline
mpl.rc('figure',figsize=(12.8,4.8))
clen.hist(bins=60)

Focando nos contigs com mais de 500 pares de base de comprimento, teremos pouco mais de 10% dos contigs (181):

In [ ]:
print(len(clen[clen > 500]))
clen[clen>500].hist(bins=150)

### 5.2 Coletando estatísticas em um _dataframe_ Pandas

Como vimos acima, podemos usar um *list comprehension*, ou seja, a notação `[ x for x in list() ]` para gerar uma lista contendo os elementos `x`. Para gerar uma tabela Pandas, podemos gerar uma lista de listas:

In [ ]:
a = []
for x in s:
    a.append([ x.id, len(x), SeqUtils.GC(x.seq) ])

No exemplo acima calculamos, simultaneamente, o comprimento dos contigs (`len(x)`) e a proporção de guaninas e citosinas (GC%), este último usando o método `GC()` do módulo __SeqUtils__, que espera um objeto Bio.Seq ou uma *string* como argumento.

A lista `a`, que contém, em cada elemento, as estatísticas para cada contig, pode ser passada para o construtor do Pandas:

In [ ]:
df = pd.DataFrame(a, columns=['id','length','gc'])

E podemos então verificar:

1. Se os nomes das colunas são os esperados

In [ ]:
df.columns

2. Se o número de linhas é igual ao número de contigs

In [ ]:
len(df)

E, finalmente, indagar quais os valores das nossas estatísticas para os dez contigs mais longos:

In [ ]:
df.sort_values(['length'], ascending=False).head()

### 5.3 Selecionando sequências com base em estatísticas

Podemos usar nosso dataframe para identificar, por exemplo, o contig mais longo:

In [ ]:
df.length.max()

In [ ]:
df[df.length == 275275]

Fica fácil ver que o contig mais longo é o 3027, com 275275 pares de bases.

__Mas onde esse contig está na lista `s`?__

Lembrando que as linhas no DataFrame `df` foram carregadas na mesma ordem que as sequências na lista `s`, podemos usar o número da linha (1711) para acessar a sequência:

In [ ]:
s[1711].id

Funciona perfeitamente mas como fazer isso __sem ficar copiando os resultados dos comando anteriores__?

Bem, nós já sabemos como encontrar a linha que nos interessa (`df[df.length == df.length.max()]`) mas o resultado mostra que o número que dá a posição da sequência em `s` é o número da linha em `df`. __Como acessar esse número__?

O número 1711 é o __índice__ da linha referente ao contig 3027 no DataFrame `df`.

O índice é um objeto `pandas.Index`, que identifica a linha mas não pode ser usado para identificar um elemento na lista `s`.

Podemos, porém, converter o índice numa coluna usando o método `tolist()`, cuja função é converter o índice unma lista que, nesse caso, terá apenas um elemento.

In [ ]:
df[df.length == df.length.max()].index.tolist()

e por fim acessamos o primeiro e único elemento dessa lista usando o índice:

In [ ]:
df[df.length == df.length.max()].index.tolist()[0]

e finalmente combinamos tudo para acessar o elemento da lista `s` que armazena o Bio.SeqRecord do contig mais longo:

In [ ]:
s[df[df.length == df.length.max()].index.tolist()[0]].id

In [ ]:
%timeit s[df[df.length == df.length.max()].index.tolist()[0]].seq

Embora ilusre bem como funciona o encadeamento de objetos e as funções do Pandas, o exemplo acima depende de como o dado foi organizado na memória do programa.

Nesse caso, termos guardado as sequências na lista `s` torna necessário todos os passos acima para acessar uma sequência de escolha mas se, aproveitando a flexibilidade do Pandas, adicionarmos as sequências a uma colunas, podemos simplificar o processo.

Como sabemos que a lista e as linhas da tabela estão na mesma ordem, o processo é simples:

In [ ]:
df['obj'] = s
df.loc[df.length == df.length.max(),'obj'].iloc[0].id

In [ ]:
%timeit df.loc[df.length == df.length.max(),'obj'].iloc[0].seq

Note que o `iloc[0]` é necessário pelo mesmo motivo que o `tolist()[0]` da opção anterior: o critério `df.length == df.length.max()` pode retornar mais de uma linha e, como desejamos um escalar

A sintaxe acima é um pouco mais elegante e fácil de ler que a anterior mas sua real vantagem aparece no significativamente menor de execução. Esse tipo de comparações será necessária sempre que o volume dos dados justificar a otimização dos procedimentos.

### 5.4 Explorando a variação no conteúdo de GC%

A função GC do módulo SeqUtils calculou a proporção de resíduos de guanina (G) e citosina (C) para cada contig.

Usando a notação de atributos, podemos extrair os valores nessa coluna como um Pandas.Series e calcular sua média:

In [ ]:
df.gc.mean()

Mais interessante ainda é comparar os valores de GC% com o comprimento dos contigs:

In [ ]:
plt.scatter(df.length,df.gc)

O gráfico acima mostra não apenas que a maioria dos contigs, especialmente os mais longos, possuem valores próximos da média mas que a variância do GC% é muito maior para os contigs curtos. Mostra também a presença de pelo menos um *outlier* (GC < 45%) entre os contigs com mais de 10000 pares de bases.

In [ ]:
df[(df.length > 10000) & (df.gc < 45)]

Por desviar da composição de bases (GC%) típica deste genoma, o contig 3043 corresponde, muito provavelmente, a uma ilha genômica ou plasmídeo, ou seja, uma região relacionada a elementos móveis e que pode ter se originado por transferência lateral, merecendo ser investigado à parte.

## 6. Analisando um genoma anotado (Biopython e Pandas)

O genoma de *K. pneumoniae* str. KPNIH39, cuja montagem analisamos nos itens anteriores, foi completamente montado e submetido ao banco de dados GenBank, de onde pode ser obtido junto com a anotação de todos os seus genes.

Vamos carregar uma cópia local do cromossomo dessa bactéria, já anotado, e ver que tipos de genes ele contém.

Primeiramente, carregamos o genoma usando BioPython (nesse caso, o arquivo GBK contém apenas uma sequência):

In [ ]:
io = SeqIO.parse("CP014762.1.gbk","genbank")
k = next(io)

E checamos seu comprimento:

In [ ]:
len(k)

Você pode verificar se esse é o comprimento esperado usando o comando `less`, em um terminal, para visualizar a primeira linha do arquivo:

`less CP014762.1.gbk`

### 6.1 Convertendo dados contidos no objeto Biopython em Pandas

O código abaixo extrai a anotação sobre genes codificantes (CDS) do objeto Bio.SeqRecord, produzido pelo SeqIO, e retorna um dataframe Pandas contendo essa informação.

Ele __não precisa ser compreendido completamente__, razão pela qual o disponibilizamos na forma de uma função.

In [ ]:
def seqrecord_to_pandas(k):
    a = []
    for f in k.features:
        if f.type == "CDS":
            locus  = f.qualifiers['locus_tag'][0]
            ftype  = f.type
            start  = f.location.start
            end    = f.location.end
            strand = f.location.strand
            if 'tanslation' in f.qualifiers:
                plen = len(f.qualifiers['translation'][0])
            else:
                plen = 0
            if 'product' in f.qualifiers:
                product = f.qualifiers['product'][0]
            else:
                product = "unannotated"
            a.append([locus,ftype,start,end,strand,plen,product])
    p = pd.DataFrame(a, columns = ['locus','type','start','end','strand','plen','product'])
    return(p)

Explicação da função `seqrecord_to_pandas` (__leitura opcional__):

Toda a anotação sobre os genes está contida no atributo `features` do objeto SeqRecord. O atributo `features` contém uma lista, cujos elementos são objetos da classe Bio.SeqFeature (variável `f`), cada qual corresponde a um gene no genoma. O atributo `qualifiers` de cada Bio.SeqFeature contém um dicionário com as anotações (função, etc) dos genes. O Bio.SeqFeature contém também um outro objeto, Bio.Location, que armazena as coordenadas do gene no genoma.

O código acima, portanto, varre uma estrutura de dados __hierárquica__, muito complexa, e representa os dados numa __forma tabular__ (Pandas), mais fácil de traballhar.

Vamos executar essa função e checar se funcionou:

In [ ]:
p = seqrecord_to_pandas(k)
p.head()

### 6.2 Estatísticas da anotação dos genes

Qual seria as dez funções mais comuns entre os genes codificantes no genoma de *K pneumoniae*?

Para responder essa pergunta, vamos contar o número de genes por função (coluna produto) usando `groupby` e contando o número de identificadores de genes (coluna *locus*):

In [ ]:
p.groupby('product').agg({'locus':'nunique'}).sort_values(['locus'], ascending=False).head()

### 6.3 Composição do cromossomo de *K. pneumoniae* KPNIH39

Na seção 5 exploramos a composição (GC%) dos contigs obtidos na montagem do genoma.

Esse genoma possue, na verdade, um cromossomo, de 5Mbp, e três plasmídeos, de tamanho entre 36000 e 285000 pares de bases.

__Questão__: qual seria o GC% do cromossomo, principal replicon desse organismo, e como se compara à composição dos contigs?

In [ ]:
SeqUtils.GC(k.seq)

O conteúdo de GC dos contigs já foi calculado, no item 5.4, usando a fórmula

In [ ]:
df.gc.mean()

A diferença entre os dois valores se deve, na maior parte, à presença dos plasmídeos entre os contigs da nossa montagem e, em menor escala, à presença de pequenos contigs que não pertencem ao genoma de *K. pneumoniae* mas permanecem nos resultados da montagem automática, como contaminação.

Apesar disso, os dois valores são muito próximos, evidenciando a qualidade da montagem.

### 6.4 GC skew do cromossomo de *K. pneumoniae* (Biopython e Matplotlib)

O cromossomo CP014762.1 é um cromossomo bacteriano circular e possui as assinaturas típicas de viéses de composição que marcam a posição dos sítios de início e término de replicação em muitas espécies de bactérias. Tais assinaturas foram extensamente estudadas por vários autores e a medida conhecida como *GC skew* e definida como a razão ($(G-C)/(G+C)$), onde as letras G e C correpondem à proporção (frequência relativa ou percentual) das bases guanina e citosina. O *GC skew* é a medida de composição do DNA mais frequentemente utilizada para evidenciar as diferenças de composição entre regiões de uma mesma molécula de DNA. Um exemplo de tais diferenças são as causadas por variações no viés mutacional das diferentes fitas do DNA (ver discussão nesse [artigo](https://www.cell.com/trends/microbiology/fulltext/S0966-842X(99)01541-3)).

Essa medida está implementada no módulo Bio.SeqUtils e pode ser calculada e visualizada, para o genoma de *K. pneumoniae* que armazenamos na variável `k`, usando o código abaixo:

In [ ]:
window = 10000
y = SeqUtils.GC_skew(k.seq,window)
x = range(1,(len(y))*window,window)
plt.plot(x,y)
plt.hlines([0.03,0.0,-0.03], xmin=0, xmax=len(k.seq), linestyles="dashed", colors=["red","black","green"])

Nesse exemplo, dividimos o cromossomo em fragmentos de 10000 pares de base (window), sem sobreposição, e calculamos o *GC skew* em cada fragmento contra a posição desse fragmento (linha azul). A linhas pontilhadas vermelha e verde indicam a posição aproximada da média do *GC skew* em cada parte do genoma.

A posição on o valor de *GC skew* passa de positiva para negativa, localizada um pouco depois da metade do comprimento do cromossomo, correponde à região do término de replicação. A região da origem também mostra uma mudança abrupta no valor do *GC skew* (diferençna entre o começo e o fim do genoma), mas a visualização não é tão clara pois a origem de replicação foi escolhida como a posição 1 do cromossomo, nessa representação linearizada.